In [1]:
# Work in progress, not a complete example.

import argparse
import gzip
import os
import sys
import time
import urllib

import numpy
import tensorflow as tf

from six.moves import xrange

In [2]:
# MNIST example (data downloading etc.) is based on Google's https://github.com/tensorflow/models/blob/master/tutorials/image/mnist/convolutional.py
# See LICENSE for details.

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
WORK_DIRECTORY = 'data'
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.

EMBEDDING_SIZE = 10
NUM_INTERNAL_CONVS = 5
NUM_UNROLL_STEPS = 10

In [3]:
def maybe_download(filename):
  """Download the data from Yann's website, unless it's already here."""
  if not tf.gfile.Exists(WORK_DIRECTORY):
    tf.gfile.MakeDirs(WORK_DIRECTORY)
  filepath = os.path.join(WORK_DIRECTORY, filename)
  if not tf.gfile.Exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
      size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
  return filepath


def extract_data(filename, num_images):
  """Extract the images into a 4D tensor [image index, y, x, channels].

  Values are rescaled from [0, 255] down to [-0.5, 0.5].
  """
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(16)
    buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
    data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
    data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    return data


def extract_labels(filename, num_images):
  """Extract the labels into a vector of int64 label IDs."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
  return labels

from collections import Counter

def error_rate(predictions, labels):
  """Return the error rate based on dense predictions and sparse labels."""
  argmax = numpy.argmax(predictions, 1)
  print(Counter(argmax).most_common())
  return 100.0 - (
      100.0 *
      numpy.sum(argmax == labels) /
      predictions.shape[0])

train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

# Generate a validation set.
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]
train_size = train_labels.shape[0]

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
def feature_extractor(input_images, training):
    x = input_images
    x = tf.layers.conv2d(x, filters=16, kernel_size=[5,5], strides=2,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=32, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=64, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=128, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, fused=True, training=training)
    x = tf.nn.relu(x)
    return x

def model_step(input_images, prior, batch_size):
    """The Model definition."""
    prior_embeddings = tf.get_variable("prior_embeddings",
                                        shape=[NUM_LABELS, EMBEDDING_SIZE],
                                        initializer=tf.random_uniform_initializer(
                                            minval=-1.0/numpy.sqrt(NUM_LABELS), maxval=1.0/numpy.sqrt(NUM_LABELS)))
    embedding_features = tf.matmul(prior, prior_embeddings)
    for i in range(4):
        embedding_features = embedding_features + tf.layers.dense(
            embedding_features, EMBEDDING_SIZE, use_bias=False, activation=tf.nn.relu)
    
    def get_dynamic_weights(weights_shape):
        num_weights = numpy.prod(weights_shape[1:])
        dynamic_weights_flat = tf.layers.dense(embedding_features, num_weights)
        dynamic_weights = tf.reshape(dynamic_weights_flat, weights_shape)
        dynamic_weights.set_shape(weights_shape)
        return dynamic_weights
    
    conv1_weights = get_dynamic_weights([batch_size, 1, 1, 128, EMBEDDING_SIZE])
    conv2_weights = [get_dynamic_weights([batch_size, 2, 2, EMBEDDING_SIZE, EMBEDDING_SIZE])
                     for _ in range(NUM_INTERNAL_CONVS)]
    conv3_weights = get_dynamic_weights([batch_size, 1, 1, EMBEDDING_SIZE, NUM_LABELS])
    conv3_biases = get_dynamic_weights([batch_size, NUM_LABELS])
    
    logits_lst, posteriors_lst = [], []
    for elm in range(batch_size):
        conv = tf.nn.conv2d([input_images[elm]],
                            conv1_weights[elm],
                            strides=[1, 1, 1, 1],
                            padding='VALID')
        relu = tf.nn.relu(conv)
        for i in range(NUM_INTERNAL_CONVS):
            conv = tf.nn.conv2d(relu,
                            conv2_weights[i][elm],
                            strides=[1, 1, 1, 1],
                            padding='SAME')
            relu = tf.nn.relu(conv) + relu
        conv = tf.nn.conv2d(relu,
                            conv3_weights[elm],
                            strides=[1, 1, 1, 1],
                            padding='VALID')
        conv_shape = conv.get_shape()
        logits = tf.nn.avg_pool(conv, ksize=[1, conv_shape[1], conv_shape[2], 1],
                                strides=[1, 1, 1, 1], padding="VALID") + conv3_biases[elm]
        logits = tf.squeeze(logits, axis=[1, 2])
        posteriors = tf.nn.softmax(logits)
        logits_lst.append(logits)
        posteriors_lst.append(posteriors)
    return tf.concat(logits_lst, axis=0), tf.concat(posteriors_lst, axis=0)

def apply(input_images, training):
    results = []
    loss = 0.0
    conv_features = feature_extractor(input_images, training=training)
    batch_size = conv_features.get_shape()[0]  # HyperNet operates on single images only
    priors = numpy.array([[1/NUM_LABELS for _ in range(NUM_LABELS)] for _ in range(batch_size)],
                         dtype=numpy.float32)
    for step in range(NUM_UNROLL_STEPS):
        logits, posteriors = model_step(conv_features, priors, batch_size)
        priors = posteriors
        results.append((logits, posteriors))
        loss += (1.0 if step + 1 == NUM_UNROLL_STEPS else 0.1) * tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=train_labels_node, logits=logits))
    return logits, loss

In [5]:
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))
eval_data = tf.placeholder(
    tf.float32,
    shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))


# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=tf.float32)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    1e-3,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)

# Predictions for the current training minibatch.
with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
    train_prediction, loss = apply(train_data_node, training=True)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,
                                                   global_step=batch)


# Predictions for the test and validation, which we'll compute less often.
with tf.variable_scope("model", reuse=True):
    eval_prediction, _ = apply(eval_data, training=False)

  # Small utility function to evaluate a dataset by feeding batches of data to
  # {eval_data} and pulling the results from {eval_predictions}.
  # Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
      raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
      end = begin + EVAL_BATCH_SIZE
      if end <= size:
        predictions[begin:end, :] = sess.run(
            eval_prediction,
            feed_dict={eval_data: data[begin:end, ...]})
      else:
        batch_predictions = sess.run(
            eval_prediction,
            feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
        predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions

  # Create a local session to run the training.
start_time = time.time()
with tf.Session() as sess:
    # Run all the initializers to prepare the trainable parameters.
    tf.global_variables_initializer().run()
    print('Initialized!')
    # Loop through training steps.
    for step in xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
      # Compute the offset of the current minibatch in the data.
      # Note that we could use better randomization across epochs.
      offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
      batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
      batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
      # This dictionary maps the batch data (as a numpy array) to the
      # node in the graph it should be fed to.
      feed_dict = {train_data_node: batch_data,
                   train_labels_node: batch_labels}
      # Run the optimizer to update weights.
      sess.run(optimizer, feed_dict=feed_dict)
      # print some extra information once reach the evaluation frequency
      if step % EVAL_FREQUENCY == 0:
        # fetch some extra nodes' data
        l, lr, predictions = sess.run([loss, learning_rate, train_prediction],
                                      feed_dict=feed_dict)
        elapsed_time = time.time() - start_time
        start_time = time.time()
        print('Step %d (epoch %.2f), %.1f ms' %
              (step, float(step) * BATCH_SIZE / train_size,
               1000 * elapsed_time / EVAL_FREQUENCY))
        print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
        print('Minibatch error: %.1f%%' % error_rate(predictions, batch_labels))
        print('Validation error: %.1f%%' % error_rate(
            eval_in_batches(validation_data, sess), validation_labels))
        sys.stdout.flush()
    # Finally print the result!
    test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
    print('Test error: %.1f%%' % test_error)


Initialized!
Step 0 (epoch 0.00), 772.7 ms
Minibatch loss: 4.361, learning rate: 0.001000
[(9, 64)]
Minibatch error: 89.1%
[(9, 5000)]
Validation error: 90.1%
Step 100 (epoch 0.12), 1015.1 ms
Minibatch loss: 2.254, learning rate: 0.001000
[(4, 10), (7, 10), (0, 9), (1, 9), (3, 8), (2, 7), (6, 6), (8, 4), (9, 1)]
Minibatch error: 21.9%
[(7, 3515), (1, 1485)]
Validation error: 82.1%
Step 200 (epoch 0.23), 923.0 ms
Minibatch loss: 1.790, learning rate: 0.001000
[(1, 11), (4, 10), (9, 9), (2, 7), (6, 7), (8, 7), (7, 6), (5, 3), (0, 2), (3, 2)]
Minibatch error: 18.8%
[(7, 2982), (1, 1233), (4, 386), (9, 295), (3, 104)]
Validation error: 74.1%
Step 300 (epoch 0.35), 927.9 ms
Minibatch loss: 1.064, learning rate: 0.001000
[(2, 9), (6, 8), (9, 8), (3, 7), (7, 7), (1, 6), (4, 6), (8, 6), (5, 4), (0, 3)]
Minibatch error: 10.9%
[(4, 977), (5, 661), (7, 655), (9, 578), (3, 559), (1, 532), (6, 387), (0, 348), (2, 301), (8, 2)]
Validation error: 25.2%
Step 400 (epoch 0.47), 937.2 ms
Minibatch loss: 

[(1, 558), (7, 552), (4, 527), (2, 520), (9, 503), (6, 496), (3, 494), (0, 489), (5, 434), (8, 427)]
Validation error: 2.2%
Step 2800 (epoch 3.26), 924.1 ms
Minibatch loss: 0.031, learning rate: 0.000857
[(1, 10), (2, 8), (4, 8), (5, 7), (7, 7), (9, 7), (0, 5), (3, 5), (6, 4), (8, 3)]
Minibatch error: 0.0%
[(1, 567), (7, 561), (4, 549), (2, 523), (6, 489), (3, 486), (9, 486), (0, 481), (5, 432), (8, 426)]
Validation error: 2.2%
Step 2900 (epoch 3.37), 927.3 ms
Minibatch loss: 0.142, learning rate: 0.000857
[(7, 10), (1, 8), (4, 7), (8, 7), (9, 7), (0, 6), (2, 6), (5, 5), (3, 4), (6, 4)]
Minibatch error: 3.1%
[(1, 567), (7, 560), (4, 534), (3, 495), (6, 488), (8, 484), (2, 480), (9, 480), (0, 478), (5, 434)]
Validation error: 1.4%
Step 3000 (epoch 3.49), 916.4 ms
Minibatch loss: 0.034, learning rate: 0.000857
[(6, 10), (3, 8), (7, 8), (5, 7), (8, 7), (1, 6), (9, 6), (2, 5), (0, 4), (4, 3)]
Minibatch error: 0.0%
[(7, 565), (1, 557), (4, 523), (3, 515), (9, 491), (2, 485), (5, 478), (0, 4

[(1, 573), (7, 559), (4, 536), (6, 495), (9, 493), (3, 485), (2, 481), (0, 480), (8, 462), (5, 436)]
Validation error: 1.1%
Step 5500 (epoch 6.40), 939.6 ms
Minibatch loss: 0.068, learning rate: 0.000735
[(0, 11), (5, 10), (2, 8), (6, 8), (3, 6), (1, 5), (8, 5), (9, 5), (4, 4), (7, 2)]
Minibatch error: 0.0%
[(1, 558), (7, 555), (9, 512), (4, 506), (6, 500), (2, 495), (3, 491), (0, 480), (8, 471), (5, 432)]
Validation error: 1.7%
Step 5600 (epoch 6.52), 920.8 ms
Minibatch loss: 0.014, learning rate: 0.000735
[(7, 9), (3, 8), (4, 8), (0, 7), (8, 7), (2, 6), (1, 5), (6, 5), (9, 5), (5, 4)]
Minibatch error: 0.0%
[(1, 566), (4, 555), (7, 552), (6, 500), (2, 489), (3, 488), (0, 479), (9, 476), (8, 465), (5, 430)]
Validation error: 1.3%
Step 5700 (epoch 6.63), 903.5 ms
Minibatch loss: 0.037, learning rate: 0.000735
[(7, 12), (0, 11), (4, 7), (1, 6), (3, 6), (2, 5), (6, 5), (8, 5), (5, 4), (9, 3)]
Minibatch error: 0.0%
[(1, 573), (7, 542), (4, 533), (0, 493), (9, 493), (3, 492), (2, 490), (6, 

[(1, 561), (7, 552), (4, 525), (9, 511), (3, 503), (6, 502), (0, 480), (2, 474), (8, 459), (5, 433)]
Validation error: 1.4%
Step 8200 (epoch 9.54), 908.6 ms
Minibatch loss: 0.030, learning rate: 0.000630
[(1, 11), (3, 8), (6, 8), (4, 7), (0, 6), (2, 6), (5, 6), (7, 4), (8, 4), (9, 4)]
Minibatch error: 0.0%
[(1, 571), (4, 549), (7, 549), (3, 518), (6, 493), (9, 491), (0, 478), (2, 468), (8, 443), (5, 440)]
Validation error: 1.7%
Step 8300 (epoch 9.66), 913.5 ms
Minibatch loss: 0.076, learning rate: 0.000630
[(1, 12), (6, 10), (0, 8), (9, 8), (2, 6), (4, 6), (3, 5), (5, 5), (7, 2), (8, 2)]
Minibatch error: 0.0%
[(1, 565), (7, 536), (4, 531), (9, 509), (6, 504), (3, 491), (0, 486), (2, 483), (8, 467), (5, 428)]
Validation error: 1.2%
Step 8400 (epoch 9.77), 908.7 ms
Minibatch loss: 0.041, learning rate: 0.000630
[(0, 10), (1, 9), (2, 8), (4, 7), (9, 7), (5, 5), (7, 5), (8, 5), (3, 4), (6, 4)]
Minibatch error: 0.0%
[(1, 564), (4, 557), (7, 538), (6, 502), (3, 496), (2, 491), (9, 484), (0, 